In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from sklearn import pipeline, linear_model, preprocessing, model_selection, metrics

/home/navarra/snap/jupyter/common/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [2]:
def predict_day(date, m_id):
    date_start = pd.to_datetime(date.strftime('%Y-%m-%d') + ' 07:00')
    date_end = pd.to_datetime(date.strftime('%Y-%m-%d') + ' 16:00')
    df = pd.DataFrame(index=pd.date_range(date_start, date_end, freq='H'))
    df['municipality_id'] = m_id
    df['hour'] = df.index.hour
    df['weekday'] = df.index.weekday
    return df

In [3]:
class Banana:
    def __init__(self, ts_params, day_predictor):
        self.ts_params = ts_params.copy()
        self.day_predictor = day_predictor
        self.tsa_models = None
        self._municipalities = None
        
    def _hourly_fit(self, data, copy=True):
        if copy:
            data = data.copy()
        data.timestamp = pd.to_datetime(data.timestamp)
        data = data.groupby([pd.Grouper(key='timestamp', freq='H'), 'municipality_id'])['usage'].max()
        data = data.reset_index()
        hazir = data.assign(hour=lambda x: x['timestamp'].dt.hour, weekday=lambda x: x['timestamp'].dt.weekday,
            daily_mean=lambda x: x.groupby([pd.Grouper(key='timestamp', freq='D'), 'municipality_id'])['usage'].transform('mean'),
            ratio_to_daily=lambda x: x['usage'] / x['daily_mean'])[['municipality_id', 'hour', 'weekday', 'ratio_to_daily']]
        
        X, y = hazir.drop('ratio_to_daily', 1), hazir['ratio_to_daily']
        self.day_predictor.fit(X, y)
        del X
        del y
        del data
        del hazir
        
    def _daily_fit(self, data, copy=True):
        df = data
        self._municipalities = df['municipality_id'].unique().tolist()
        self.tsa_models = {}
        if copy:
            df = df.copy()
        df['timestamp'] = pd.to_datetime(df['timestamp'])

        df = df.groupby(['municipality_id', pd.Grouper(key='timestamp', freq='H')])['usage'].max()
        df = df.sort_index()
        for muni in self._municipalities:
            dm = df[muni].resample('D').mean()
            dm_filled = dm.fillna(dm.groupby([lambda x: x.month, lambda x:x.weekday]).transform('mean'))
            self.tsa_models[muni] = sm.tsa.SARIMAX(dm_filled, **self.ts_params).fit()
            
    def fit(self, data, copy=True):
        self._daily_fit(data=data, copy=copy)
        self._hourly_fit(data=data, copy=copy)
        return self
    
    def predict(self, date_start, date_end, municipality_id):
        preds = self.tsa_models[municipality_id].predict(date_start, date_end)
        dfs = []
        for i, mean in zip(preds.index, preds):
            dayframe = predict_day(i, municipality_id)
            dayframe['ratio'] = self.day_predictor.predict(dayframe)
            dayframe['usage'] = dayframe['ratio'] * mean
            dayframe.drop('ratio', 1, inplace=True)
            dfs.append(dayframe)
        return pd.concat(dfs, axis=0)
    
    def __repr__(self):
        return f"{self.__class__.__name__}(ts_params={self.ts_params}, day_predictor={self.day_predictor})"
        
    

In [4]:
p = pipeline.Pipeline([
    ('encode', preprocessing.OneHotEncoder()),
    ('poly', preprocessing.PolynomialFeatures(2)),
    ('regress', linear_model.LinearRegression())
    ])

In [5]:
baseline = Banana(dict(order=(1,0,0)), p)
canavar = Banana(dict(order=(1,1,1), seasonal_order=(1,1,1,7)), p)

In [6]:
data = pd.read_csv('municipality_bus_utilization.csv')
data.timestamp = pd.to_datetime(data.timestamp)

In [7]:
test_cutoff = '2017-08-05'
data_ub = '2017-08-19'

In [8]:
baseline.fit(data.loc[lambda x: x.timestamp < test_cutoff])

Banana(ts_params={'order': (1, 0, 0)}, day_predictor=Pipeline(steps=[('encode', OneHotEncoder()), ('poly', PolynomialFeatures()),
                ('regress', LinearRegression())]))

In [9]:
canavar.fit(data.loc[lambda x: x.timestamp < test_cutoff])

/home/navarra/snap/jupyter/common/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:965: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/home/navarra/snap/jupyter/common/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


Banana(ts_params={'order': (1, 1, 1), 'seasonal_order': (1, 1, 1, 7)}, day_predictor=Pipeline(steps=[('encode', OneHotEncoder()), ('poly', PolynomialFeatures()),
                ('regress', LinearRegression())]))

In [10]:
def resample_hours(df, copy=True):
    if copy:
        df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    df = df.groupby(['municipality_id', pd.Grouper(key='timestamp', freq='H')])['usage'].max()
    df = df.sort_index()
    return df

In [11]:
bp = baseline.predict(test_cutoff, data_ub, 2).assign(actuals=resample_hours(data)[2][test_cutoff:])

In [12]:
cp = canavar.predict(test_cutoff, data_ub, 2).assign(actuals=resample_hours(data)[2][test_cutoff:])

In [13]:
tum = cp.assign(bp=bp['usage']).dropna()

In [14]:
for muni in range(10):
    bp = baseline.predict(test_cutoff, data_ub, muni).assign(actuals=resample_hours(data)[muni][test_cutoff:])
    cp = canavar.predict(test_cutoff, data_ub, muni).assign(actuals=resample_hours(data)[muni][test_cutoff:])
    tum = cp.assign(bp=bp['usage']).dropna()
    print('Vilayet', muni)
    bmse = metrics.mean_squared_error(tum['actuals'], tum['bp'])**0.5
    bmae = metrics.mean_absolute_error(tum['actuals'], tum['bp'])
    bmape = metrics.mean_absolute_percentage_error(tum['actuals'], tum['bp']) * 100.
    cmse = metrics.mean_squared_error(tum['actuals'], tum['usage'])**0.5
    cmae = metrics.mean_absolute_error(tum['actuals'], tum['usage'])
    cmape = metrics.mean_absolute_percentage_error(tum['actuals'], tum['usage']) * 100.
    print("Baseline RMSE:", bmse.round(2))
    print("Gelismis RMSE:", cmse.round(2))
    print()
    print("Baseline MAE:", bmae.round(2))
    print("Gelismis MAE:", cmae.round(2))
    print()
    print("Baseline MAPE:", bmape.round(2))
    print("Gelismis MAPE:", cmape.round(2))
    print('----------------------------------------')

Vilayet 0
Baseline RMSE: 396.3
Gelismis RMSE: 337.97

Baseline MAE: 265.28
Gelismis MAE: 192.97

Baseline MAPE: 32.58
Gelismis MAPE: 21.55
----------------------------------------
Vilayet 1
Baseline RMSE: 60.56
Gelismis RMSE: 39.25

Baseline MAE: 53.03
Gelismis MAE: 31.91

Baseline MAPE: 15.82
Gelismis MAPE: 9.93
----------------------------------------
Vilayet 2
Baseline RMSE: 80.08
Gelismis RMSE: 73.04

Baseline MAE: 63.08
Gelismis MAE: 59.62

Baseline MAPE: 13.4
Gelismis MAPE: 11.47
----------------------------------------
Vilayet 3
Baseline RMSE: 451.57
Gelismis RMSE: 298.93

Baseline MAE: 397.34
Gelismis MAE: 175.31

Baseline MAPE: 32.99
Gelismis MAPE: 22.21
----------------------------------------
Vilayet 4
Baseline RMSE: 1181.37
Gelismis RMSE: 738.29

Baseline MAE: 1033.91
Gelismis MAE: 443.95

Baseline MAPE: 38.36
Gelismis MAPE: 25.61
----------------------------------------
Vilayet 5
Baseline RMSE: 145.84
Gelismis RMSE: 272.24

Baseline MAE: 110.08
Gelismis MAE: 226.3

Baselin